In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('../input/coleridge-initiative-preprocessed-to-csv-data/df_train_publications.csv')
test = pd.read_csv('../input/coleridge-initiative-preprocessed-to-csv-data/df_test_publications.csv')
labels = pd.read_csv('../input/coleridge-initiative-preprocessed-to-csv-data/labels.csv')
train_meta = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')

def concat(column):
    res = ' '
    for st in column:
        if type(st) == str:
            res += st
    return res

train_gr = train.groupby('pub_id')['text'].apply(concat).reset_index()
train_gr.loc[train['pub_id'].isin(train_meta['Id']), 'cleaned_label'] = train_meta.loc[train_meta['Id'].isin(train_gr['pub_id']),'cleaned_label']

## NER Feature extraction

We will use spacy pre-trained model to extract some features.

In [ ]:
# !python3 -m spacy download en_core_web_lg
import spacy
sp_lg = spacy.load('en_core_web_lg')

document = train_gr.loc[0, 'text']
print({(ent.text.strip(), ent.label_) for ent in sp_lg(document).ents})

Here we have to decide what entities are important for us, for example, organizations, and go through the train_gr with the loop.

## Count Vectorizer and dimensionality reduction

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
corpus = train_gr['text']

X = vectorizer.fit_transform(corpus)

In [ ]:
X.shape

In [ ]:
vectorizer_label = CountVectorizer()
labels_corpus = train_gr['cleaned_label']
y = vectorizer_label.fit_transform(labels_corpus)

y.shape

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, n_iter=7, random_state=42)
svd.fit(X)
X_tr = svd.transform(X)

svd_y = TruncatedSVD(n_components=20, n_iter=7, random_state=42)
svd_y.fit(y)
y_tr = svd_y.transform(y)

In [ ]:
X_tr

In [ ]:
y_tr

In [ ]:
X_tr.shape

In [ ]:
y_tr.shape

Also we can use, for example, TfIdf Vectorizer.

## Some pre-trained tokenizers

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [ ]:
encoded_input = tokenizer(train_gr.loc[0, 'text'])
print(encoded_input)

## Gensim Word2Vec

In [ ]:
import gensim
w2v_model = gensim.models.Word2Vec(train_gr.loc[:10, 'text'].str.split().to_list(), size=100, min_count=1, window=5, iter=50)

In [ ]:
w2v_model.train(train_gr.loc[:10, 'text'].str.split().to_list(), total_examples=1, epochs=50)

In [ ]:
train_gr.loc[0, 'text']

In [ ]:
w2v_model.wv['DOT', 'Lebanon\'s' ,'ICT' ,'training', 'program']